generate pointcloud (original one)

In [10]:
import os
import cv2
import open3d as o3d

# Define directories and camera intrinsics
directories = {
    "front": {
        "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_red45/bag_images/rgb_mask",
        "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_red45/bag_images/depth_mask",
        "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_red45/pointcloud",
        "intrinsics": (1070.1000, 1070.4301, 939.8100, 549.6520)
    # },
    # "top": {
    #     "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/fold_red45/bag_top_images/rgb_mask",
    #     "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/fold_red45/bag_top_images/depth_mask",
    #     "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/fold_red45/pointcloud_top",
    #     "intrinsics": (1060.4000, 1060.0100, 946.0600, 496.6630)
    # },
    # "side": {
    #     "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/bag_side_images/rgb",
    #     "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/bag_side_images/depth",
    #     "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_side_cp",
    #     "intrinsics": (1066.4000, 1065.9900, 970.8000, 547.1950)
    }}


# Function to process images and generate point clouds
def process_images_and_generate_pointclouds(rgb_dir, depth_dir, point_dir, intrinsics):
    if not os.path.exists(point_dir):
        os.makedirs(point_dir)

    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_files = sorted([f for f in os.listdir(depth_dir) if f.endswith('.png')])
    pointcloud_count = 0

    for rgb_file, depth_file in zip(rgb_files, depth_files):
        rgb_image_path = os.path.join(rgb_dir, rgb_file)
        depth_image_path = os.path.join(depth_dir, depth_file)
        
        #color_raw = o3d.io.read_image(rgb_image_path)
        #depth_raw = o3d.io.read_image(depth_image_path)
        color_raw = cv2.imread(rgb_image_path)
        depth_raw = cv2.imread(depth_image_path,cv2.IMREAD_UNCHANGED)
        # print(f"Original RGB range: {color_raw.min()} - {color_raw.max()}")
        # print(f"Original Depth range: {depth_raw.min()} - {depth_raw.max()}")
        


        depth_hd = o3d.geometry.Image(depth_raw)
        color_hd = o3d.geometry.Image(color_raw)

       
        
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_hd, depth_hd, convert_rgb_to_intensity=False)
        fx, fy, cx, cy = intrinsics
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(
                width=1920,
                height=1080,
                fx=fx, fy=fy,
                cx=cx, cy=cy
            )
        )
        pcd.transform([[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        pointcloud_path = os.path.join(point_dir, rgb_file.replace('.png', '.pcd'))
        o3d.io.write_point_cloud(pointcloud_path, pcd)
        pointcloud_count += 1

    print(f"Total number of point clouds generated for {point_dir}: {pointcloud_count}")

# Process each set of directories
for key, value in directories.items():
    process_images_and_generate_pointclouds(value["rgb"], value["depth"], value["point"], value["intrinsics"])


Total number of point clouds generated for /home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_red45/pointcloud: 91


generate pointcloud resized

In [9]:
import os
import cv2
import open3d as o3d

# Define directories, camera intrinsics, and new directories for resized images
directories = {
    "front": {
        "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/napkin_pink45/bag_images_very_crumple/rgb_mask",
        "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/napkin_pink45/bag_images_very_crumple/depth_mask",
        "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/napkin_pink45/bag_images_very_crumple/pointcloud",
        "intrinsics": (1070.1000, 1070.4301, 939.8100, 549.6520)}
    #     ,
    # "top": {
    #     "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/bag_top_images/rgb_mask",
    #     "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/bag_top_images/depth_mask",
    #     "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_top",
    #     "intrinsics": (1060.4000, 1060.0100, 946.0600, 496.6630)},
    # "side": {
    #     "rgb": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/bag_side_images/rgb_mask",
    #     "depth": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/bag_side_images/depth_mask",
    #     "point": "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_side",
    #     "intrinsics": (1066.4000, 1065.9900, 970.8000, 547.1950)}   
}

def process_images_and_generate_pointclouds(value):
    rgb_dir = value["rgb"]
    depth_dir = value["depth"]
    point_dir = value["point"]
    intrinsics = value["intrinsics"]

    if not os.path.exists(point_dir):
        os.makedirs(point_dir)


    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_files = sorted([f for f in os.listdir(depth_dir) if f.endswith('.png')])
    pointcloud_count = 0

    for rgb_file, depth_file in zip(rgb_files, depth_files):
        rgb_image_path = os.path.join(rgb_dir, rgb_file)
        depth_image_path = os.path.join(depth_dir, depth_file)
        
        color_raw = cv2.imread(rgb_image_path)
        color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
        depth_raw = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)
        
        print(f"Original RGB range: {color_raw.min()} - {color_raw.max()}")
        print(f"Original Depth range: {depth_raw.min()} - {depth_raw.max()}")
        
        color_hd = cv2.resize(color_raw, (1920, 1080), interpolation=cv2.INTER_CUBIC)
        depth_hd = cv2.resize(depth_raw, (1920, 1080), interpolation=cv2.INTER_NEAREST)
        
        print(f"Resized RGB range: {color_hd.min()} - {color_hd.max()}")
        print(f"Resized Depth range: {depth_hd.min()} - {depth_hd.max()}")
        # Save the resized images
        # resized_rgb_path = os.path.join(resized_rgb_dir, rgb_file)
        # resized_depth_path = os.path.join(resized_depth_dir, depth_file)
        # cv2.imwrite(resized_rgb_path, color_hd)
        # cv2.imwrite(resized_depth_path, depth_hd)

        # Continue processing for point clouds as before
        depth_hd = o3d.geometry.Image(depth_hd)
        color_hd = o3d.geometry.Image(color_hd)

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_hd, depth_hd, convert_rgb_to_intensity=False)
        fx, fy, cx, cy = intrinsics
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(width=1920, height=1080, fx=fx, fy=fy, cx=cx, cy=cy)
        )
        pcd.transform([[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        pointcloud_path = os.path.join(point_dir, rgb_file.replace('.png', '.pcd'))
        o3d.io.write_point_cloud(pointcloud_path, pcd)
        pointcloud_count += 1

    print(f"Total number of point clouds generated for {point_dir}: {pointcloud_count}")

# Process each set of directories
for key, value in directories.items():
    process_images_and_generate_pointclouds(value)


Original RGB range: 0 - 255
Original Depth range: 0 - 951
Resized RGB range: 0 - 255
Resized Depth range: 0 - 951
Total number of point clouds generated for /home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/napkin_pink45/bag_images_very_crumple/pointcloud: 1


visual just point cloud + tracker

In [108]:
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os

def set_camera_view(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1070.1000, fy=939.8100, cx=960.0, cy=549.6520)
    camera_parameters.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)





def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

index = 30
# pcd_directory_path_zed_mask = "/media/ubb/4T/human_demonstration/grasp_point_all_sample/lsshirt_bluedot/pointcloud"
# pcd_directory_path_zedtop_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_top"
# pcd_directory_path_zedside_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_side"
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_cp"
pcd_directory_path_zedtop = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_top_cp"
pcd_directory_path_zedside = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_side_cp"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)

pcd_files_top = sorted([file for file in os.listdir(pcd_directory_path_zedtop) if file.endswith('.pcd')])
pcd_file_path_top = os.path.join(pcd_directory_path_zedtop, pcd_files_top[index])  # Index 32 for the 33rd file
pcd_zed_top = o3d.io.read_point_cloud(pcd_file_path_top)

pcd_files_side = sorted([file for file in os.listdir(pcd_directory_path_zedside) if file.endswith('.pcd')])
pcd_file_path_side = os.path.join(pcd_directory_path_zedside, pcd_files_side[index])  # Index 32 for the 33rd file
pcd_zed_side = o3d.io.read_point_cloud(pcd_file_path_side)

# pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
# pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
# pcd_zed = o3d.io.read_point_cloud(pcd_file_path)
# print(pcd_file_path)

vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

translation_zed = [0.139, -0.68, -1.52] # front modified leoa
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
# translation_zedtop = [-0.508, -0.308, -0.84] # top modified leo
# quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo

translation_zedtop_ryd = [-0.471, -0.308, -0.81] # top modified leo
quaternion_zedtop_ryd = [0.655,0.611, 0.305, 0.323] # top modified leo

# translation_zedside = [1.0, -1.030, -0.01]  # side modified leo
# quaternion_zedside =  [0.385, -0.687, -0.588,  0.186]

translation_zedside_ryd = [0.99, -1.03, 0.05]  # side modified leo
quaternion_zedside_ryd =  [0.408, -0.662, -0.602,  0.182]

transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop, quaternion_zedtop_ryd)
transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop_ryd, quaternion_zedtop_ryd)
# transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside, quaternion_zedside)
transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside_ryd, quaternion_zedside_ryd)
# transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)

vis.add_geometry(transformed_pcd_zed)
# vis.add_geometry(transformed_pcd_zedtop)
vis.add_geometry(transformed_pcd_zedside)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()

from here is the code to generate the grasping point

In [2]:
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os


def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd
 

def read_tracker_data(file_path):
    """Read the tracker data from a file and return the position and quaternion."""
    positions = []
    quaternions = []
    grippers = []
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            # if i % 20 == 0:  
            pos = ast.literal_eval(row['position'])
            ori = ast.literal_eval(row['orientation'])
            gripper = int(row['gripper'])
            positions.append(np.array([pos['x'], pos['y'], pos['z']]))
            quaternions.append([
                round(ori['w'],3),
                round(ori['x'],3), 
                round(ori['y'],3),
                round(ori['z'],3)])
            grippers.append(gripper)
    return positions, quaternions, grippers

def set_camera_view(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1070.1000, fy=939.8100, cx=960.0, cy=549.6520)
    camera_parameters.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)

def set_camera_view_top(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1060.4000, fy=1060.0100, cx=946.0600, cy=496.6630)
    camera_parameters.extrinsic = np.array([
        [-1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)


In [3]:
tracker1_position, tracker1_quaternion, tracker1_gripper = read_tracker_data("/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/bag/tracker_1/tracker_right_zed_modified.csv")
tracker4_position, tracker4_quaternion, tracker4_gripper= read_tracker_data("/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/bag/tracker_4/tracker_left_zed_modified.csv")

In [4]:
def calculate_extended_point(position, quaternion, extensin_length_x, extension_length_y, extension_length_z):
    R = o3d.geometry.get_rotation_matrix_from_quaternion(quaternion)
    extension_vector = np.array([extensin_length_x, extension_length_y, extension_length_z])
    extended_vector = np.dot(R, extension_vector)
    extended_point = np.array(position) + extended_vector
    return extended_point

In [7]:
index = 40
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud"
# pcd_directory_path_zedtop = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_top"
# pcd_directory_path_zedside = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_side"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)
print(pcd_file_path)

vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

# translation_zedside = [1.00, -0.988, -0.005] # original
# translation_zedside = [0.98, -1.045, -0.0075] # side modified lp
# translation_zedside = [1.05, -0.955, -0.04] # side modified leo
# quaternion_zedside =  [0.402, -0.666, -0.590, 0.215]
# translation_zed = [0.137, -0.69, -1.52]
# quaternion_zed =  [0.689, 0.128, 0.086, -0.708]
# translation_zed = [0.12, -0.70, -1.52] # front original
# translation_zed = [0.125, -0.71, -1.525] # front modified lp
# quaternion_zed =  [0.6837931, 0.12103661, 0.08102451, -0.71521635] # front original
# translation_zed = [0.135, -0.70, -1.52] # front modified
# quaternion_zed = [0.691, 0.122, 0.080, -0.708] # front modified
translation_zed = [0.139, -0.68, -1.52] # front modified leo
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
translation_zedtop = [-0.508, -0.308, -0.84] # top modified leo
quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
translation_zedside = [1.0, -0.998, 0.00] # side modified leo
quaternion_zedside =  [  0.389, -0.683, -0.587,  0.193] 



# transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed, translation_zedtop, quaternion_zedtop)
transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
vis.add_geometry(transformed_pcd_zed)


colors = {0: [1, 0, 0], 1: [128/255, 0, 128/255]} 
color_tracker1 = colors[tracker1_gripper[index]]
color_tracker4 = colors[tracker4_gripper[index]]

sphere3 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
sphere3.translate(tracker1_position[index]) 
sphere3.paint_uniform_color(color_tracker1) 
vis.add_geometry(sphere3)
frame3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
rotation_matrix3 = o3d.geometry.get_rotation_matrix_from_quaternion(tracker1_quaternion[index])
frame3.rotate(rotation_matrix3, center=[0, 0, 0])
frame3.translate(tracker1_position[index])
vis.add_geometry(frame3)

sphere4 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
sphere4.translate(tracker4_position[index]) 
sphere4.paint_uniform_color(color_tracker4)
vis.add_geometry(sphere4)
frame4 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
rotation_matrix4 = o3d.geometry.get_rotation_matrix_from_quaternion(tracker4_quaternion[index])
frame4.rotate(rotation_matrix4, center=[0, 0, 0])
frame4.translate(tracker4_position[index])
vis.add_geometry(frame4)


# sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
# sphere.translate([ 0.15318992, -0.39948571, -0.75567545])
# sphere_size = 0.02
# sphere.paint_uniform_color([1, 1, 0]) 
# vis.add_geometry(sphere)

if tracker1_gripper[index] == 1:  # Assuming 1 is open and uses purple color right
#     # extended_point = calculate_extended_point(tracker1_position[index], tracker1_quaternion[index],-0.025, 0.075, 0.11)
    extended_point = calculate_extended_point(tracker1_position[index], tracker1_quaternion[index],0.02, 0.045, 0.10)
    print(extended_point)
    pcd_points = np.asarray(transformed_pcd_zed.points)
    distances = np.linalg.norm(pcd_points - extended_point, axis=1)
    closest_point_index = np.argmin(distances)
    closest_point = pcd_points[closest_point_index]

    highlight_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
    highlight_sphere.translate(closest_point)
    highlight_sphere.paint_uniform_color([1, 1, 0])  # Using yellow to highlight
    vis.add_geometry(highlight_sphere)

if tracker4_gripper[index] == 1:  # Assuming 1 is open and uses purple color left
    # extended_point = calculate_extended_point(tracker4_position[index], tracker4_quaternion[index],-0.025, 0.075, 0.11)
    extended_point = calculate_extended_point(tracker4_position[index], tracker4_quaternion[index],-0.02, 0.045, 0.10)
    print(extended_point)
    pcd_points = np.asarray(transformed_pcd_zed.points)
    distances = np.linalg.norm(pcd_points - extended_point, axis=1)
    closest_point_index = np.argmin(distances)
    closest_point = pcd_points[closest_point_index]

    highlight_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
    highlight_sphere1.translate(closest_point)
    highlight_sphere1.paint_uniform_color([1, 1, 0]) 
    vis.add_geometry(highlight_sphere1) # Using yellow to highlight    vis.add_geometry(highlight_sphere)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()

/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud/1719056741583826897.pcd
[ 0.03582356 -0.54682094 -0.63842815]


to check the grasping point position is right

In [ ]:
import pandas as pd

index = 172
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud"
# pcd_directory_path_zedtop = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_top"
# pcd_directory_path_zedside = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_side"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)
print(pcd_file_path)

vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

# translation_zedside = [1.00, -0.988, -0.005] # original
# translation_zedside = [0.98, -1.045, -0.0075] # side modified lp
# translation_zedside = [1.05, -0.955, -0.04] # side modified leo
# quaternion_zedside =  [0.402, -0.666, -0.590, 0.215]
# translation_zed = [0.137, -0.69, -1.52]
# quaternion_zed =  [0.689, 0.128, 0.086, -0.708]
# translation_zed = [0.12, -0.70, -1.52] # front original
# translation_zed = [0.125, -0.71, -1.525] # front modified lp
# quaternion_zed =  [0.6837931, 0.12103661, 0.08102451, -0.71521635] # front original
# translation_zed = [0.135, -0.70, -1.52] # front modified
# quaternion_zed = [0.691, 0.122, 0.080, -0.708] # front modified
translation_zed = [0.139, -0.68, -1.52] # front modified leo
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
translation_zedtop = [-0.508, -0.308, -0.84] # top modified leo
quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
translation_zedside = [1.0, -0.998, 0.00] # side modified leo
quaternion_zedside =  [  0.389, -0.683, -0.587,  0.193] 



# transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed, translation_zedtop, quaternion_zedtop)
transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
vis.add_geometry(transformed_pcd_zed)


colors = {0: [1, 0, 0], 1: [128/255, 0, 128/255]} 
color_tracker1 = colors[tracker1_gripper[index]]
color_tracker4 = colors[tracker4_gripper[index]]

sphere3 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
sphere3.translate(tracker1_position[index]) 
sphere3.paint_uniform_color(color_tracker1) 
vis.add_geometry(sphere3)
frame3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
rotation_matrix3 = o3d.geometry.get_rotation_matrix_from_quaternion(tracker1_quaternion[index])
frame3.rotate(rotation_matrix3, center=[0, 0, 0])
frame3.translate(tracker1_position[index])
vis.add_geometry(frame3)

sphere4 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
sphere4.translate(tracker4_position[index]) 
sphere4.paint_uniform_color(color_tracker4)
vis.add_geometry(sphere4)
frame4 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
rotation_matrix4 = o3d.geometry.get_rotation_matrix_from_quaternion(tracker4_quaternion[index])
frame4.rotate(rotation_matrix4, center=[0, 0, 0])
frame4.translate(tracker4_position[index])
vis.add_geometry(frame4)


# sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
# sphere.translate([ 0.15318992, -0.39948571, -0.75567545])
# sphere_size = 0.02
# sphere.paint_uniform_color([1, 1, 0]) 
# vis.add_geometry(sphere)
csv_path = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/gripper_changes.csv"

df = pd.read_csv(csv_path)

grasp_position = eval(df.iloc[1]['grasp_position'])

if tracker1_gripper[index] == 1:  # Assuming 1 is open and uses purple color right
#     # extended_point = calculate_extended_point(tracker1_position[index], tracker1_quaternion[index],-0.025, 0.075, 0.11)
    # extended_point = calculate_extended_point(tracker1_position[index], tracker1_quaternion[index],0.02, 0.045, 0.10)
    # print(extended_point)
    pcd_points = np.asarray(transformed_pcd_zed.points)
    distances = np.linalg.norm(pcd_points - grasp_position, axis=1)
    closest_point_index = np.argmin(distances)
    closest_point = pcd_points[closest_point_index]

    highlight_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
    highlight_sphere.translate(closest_point)
    highlight_sphere.paint_uniform_color([1, 1, 0])  # Using yellow to highlight
    vis.add_geometry(highlight_sphere)

if tracker4_gripper[index] == 1:  # Assuming 1 is open and uses purple color left
    # extended_point = calculate_extended_point(tracker4_position[index], tracker4_quaternion[index],-0.025, 0.075, 0.11)
    # extended_point = calculate_extended_point(tracker4_position[index], tracker4_quaternion[index],-0.02, 0.045, 0.10)
    # print(extended_point)
    pcd_points = np.asarray(transformed_pcd_zed.points)
    distances = np.linalg.norm(pcd_points - grasp_position, axis=1)
    closest_point_index = np.argmin(distances)
    closest_point = pcd_points[closest_point_index]

    highlight_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
    highlight_sphere1.translate(closest_point)
    highlight_sphere1.paint_uniform_color([1, 1, 0]) 
    vis.add_geometry(highlight_sphere1) # Using yellow to highlight    vis.add_geometry(highlight_sphere)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()

In [11]:
from scipy.spatial.transform import Rotation as R
import os
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os
import copy

def find_nearest_timestamp(target_timestamp, timestamps):
    timestamps = np.array(timestamps, dtype=np.int64)
    idx = np.abs(timestamps - target_timestamp).argmin()
    return timestamps[idx]

def load_point_cloud(file_path):
    return o3d.io.read_point_cloud(file_path)

def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd


base_dir ='/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_merge'
pointcloud_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud"
pointcloud_top_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_top"
pointcloud_side_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_side"
    
pointcloud_files = sorted(os.listdir(pointcloud_folder))
pointcloud_top_files = sorted(os.listdir(pointcloud_top_folder))
pointcloud_side_files = sorted(os.listdir(pointcloud_side_folder))



for index in range(len(pointcloud_files)):
# for pointcloud_file in posintcloud_files:
    pointcloud_file = pointcloud_files[index]
    pointcloud_top_file = pointcloud_top_files[index]
    pointcloud_side_file = pointcloud_side_files[index]
       
    pointcloud_path = os.path.join(pointcloud_folder, pointcloud_file)
    # top_pointcloud_path = os.path.join(pointcloud_top_folder, f"{nearest_top_timestamp}.pcd")
    # side_pointcloud_path = os.path.join(pointcloud_side_folder, f"{nearest_side_timestamp}.pcd")
    top_pointcloud_path = os.path.join(pointcloud_top_folder, pointcloud_top_file)
    side_pointcloud_path = os.path.join(pointcloud_side_folder, pointcloud_side_file)

    front_pcd = load_point_cloud(pointcloud_path)
    top_pcd = load_point_cloud(top_pointcloud_path)
    side_pcd = load_point_cloud(side_pointcloud_path)


    translation_zed = [0.139, -0.68, -1.525] # front modified leo final
    # quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
    quaternion_zed =  [0.690, 0.126, 0.084, -0.708]  # front modified leo final


    translation_zedtop = [-0.508, -0.313, -0.84] # top modified leo final
    # quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
    quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo final

    translation_zedside = [1.0, -1.030, -0.01] # side modified leo final
    # quaternion_zedside =  [0.392,  -0.691, -0.578,  0.195] # side modified leo
    # quaternion_zedside =  [  0.389, -0.683, -0.587,  0.193] 
    quaternion_zedside =  [0.385, -0.687, -0.588,  0.186]  # side modified leo final


    transformed_pcd_zed = transform_point_cloud(front_pcd, translation_zed, quaternion_zed)
    transformed_pcd_zedtop = transform_point_cloud(top_pcd, translation_zedtop, quaternion_zedtop)
    transformed_pcd_zedside = transform_point_cloud(side_pcd, translation_zedside, quaternion_zedside)

    # merged_point_cloud2 = transformed_pcd_zed + transformed_pcd_zedtop 
    # merged_point_cloud3 = transformed_pcd_zed + transformed_pcd_zedtop + transformed_pcd_zedside
    output_path = os.path.join(base_dir, f"{pointcloud_file}")
    o3d.io.write_point_cloud(output_path, merged_point_cloud3)

